# Multi-Agent PID Control - Entrenamiento y Visualización

Este notebook prueba la arquitectura multi-agente para control PID adaptativo.

## 1. Imports y Configuración

In [ ]:
# Imports del proyecto
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List

# Arquitectura multi-agente
from multi_agent_env_modular import MultiAgentPIDEnv

# Ambientes
from Environment.simulation_env import SimulationPIDEnv
from Simuladores.tanque_simple import TankSimulator

print("✅ Imports completados")

## 2. Configuración del Experimento

In [ ]:
# Configuración para MODO DIRECTO (más simple para empezar)
config = {
    # Modo de operación
    'mode': 'direct',
    
    # Variables
    'n_manipulable_vars': 2,  # 2 variables a controlar
    'n_variables': 2,
    
    # Entrenamiento
    'n_episodes': 50,  # Episodios por agente (empezar con pocos)
    'j_max_retries': 3,  # Reintentos de validación
    
    # Parámetros del ambiente
    'upper_range': [100.0, 90.0],
    'lower_range': [0.0, 0.0],
    'setpoint': [50.0, 45.0],
    'dead_band': [2.0, 2.0],
    'dt': 1.0,
    'max_episode_steps': 100,
    
    # Parámetros de agentes DQN
    'agent_lr': 0.001,
    'agent_gamma': 0.99,
    'epsilon_start': 1.0,
    'epsilon_min': 0.01,
    'epsilon_decay': 0.995,
    'initial_pid': (1.0, 0.1, 0.05),
    'device': 'cpu'  # Cambiar a 'cuda' si hay GPU
}

print("Configuración:")
print(f"  Modo: {config['mode']}")
print(f"  Variables: {config['n_manipulable_vars']}")
print(f"  Episodios: {config['n_episodes']}")
print(f"  Reintentos: {config['j_max_retries']}")

## 3. Crear Ambiente Multi-Agente

In [ ]:
# Crear ambiente multi-agente
multi_env = MultiAgentPIDEnv(config)

# Conectar simulador externo (opcional)
# tank = TankSimulator(area=1.0, cv=0.1, max_height=10.0, max_flow_in=0.5, dt=1.0)
# multi_env.base_env.connect_external_process(tank)

print("\n✅ Ambiente multi-agente creado")

## 4. Entrenamiento

In [ ]:
# Entrenar agentes
print("\n" + "="*80)
print("INICIANDO ENTRENAMIENTO")
print("="*80)

best_pids, best_setpoints = multi_env.train()

print("\n" + "="*80)
print("ENTRENAMIENTO FINALIZADO")
print("="*80)
print(f"\nPIDs optimizados:")
for i, pid in enumerate(best_pids):
    print(f"  Variable {i}: Kp={pid[0]:.4f}, Ki={pid[1]:.4f}, Kd={pid[2]:.4f}")

## 5. Evaluación y Recolección de Datos

In [ ]:
def evaluate_pid(env, pid, var_idx, setpoint, n_steps=100):
    """
    Evaluar un PID en el ambiente y recolectar datos.
    
    Returns:
        dict con 'pv', 'sp', 'error', 'actions'
    """
    # Configurar ambiente
    env.set_setpoint(setpoint, var_idx=var_idx)
    env.pid_action_space.set_pid(pid[0], pid[1], pid[2])
    
    # Reset
    obs, info = env.reset()
    
    # Recolectar datos
    trajectory = {
        'pv': [],
        'sp': [],
        'error': [],
        'actions': []
    }
    
    for step in range(n_steps):
        # Acción 6 = mantener PID
        obs, reward, terminated, truncated, info = env.step(6)
        
        # Guardar datos
        trajectory['pv'].append(obs[0])
        trajectory['sp'].append(obs[1])
        trajectory['error'].append(obs[2])
        
        if 'control_outputs' in info:
            trajectory['actions'].append(info['control_outputs'][var_idx])
        else:
            trajectory['actions'].append(0.0)
        
        if terminated or truncated:
            break
    
    return trajectory

# Evaluar cada agente
print("\nEvaluando PIDs optimizados...")
trajectories = []

for i in range(config['n_manipulable_vars']):
    print(f"  Evaluando Variable {i}...")
    traj = evaluate_pid(
        env=multi_env.base_env,
        pid=best_pids[i],
        var_idx=i,
        setpoint=config['setpoint'][i],
        n_steps=100
    )
    trajectories.append(traj)

print("✅ Evaluación completada")

## 6. Visualización - PV vs SP por Agente

In [ ]:
# Crear gráfico de PV vs SP para cada agente
n_agents = config['n_manipulable_vars']

fig, axes = plt.subplots(n_agents, 1, figsize=(12, 5*n_agents))

# Si solo hay un agente, convertir a lista
if n_agents == 1:
    axes = [axes]

for i in range(n_agents):
    ax = axes[i]
    traj = trajectories[i]
    dead_band = config['dead_band'][i]
    
    # PV y Setpoint
    ax.plot(traj['pv'], 'b-', label='PV (Nivel)', linewidth=2)
    ax.plot(traj['sp'], 'r--', label='Setpoint', linewidth=2)
    
    # Banda muerta
    ax.fill_between(
        range(len(traj['pv'])),
        [s - dead_band for s in traj['sp']],
        [s + dead_band for s in traj['sp']],
        alpha=0.2,
        color='green',
        label='Dead Band'
    )
    
    # Configuración
    ax.set_xlabel('Step', fontsize=12)
    ax.set_ylabel('Valor', fontsize=12)
    ax.set_title(
        f'Agente Controlador {i} - PV vs Setpoint\n'
        f'PID: Kp={best_pids[i][0]:.3f}, Ki={best_pids[i][1]:.3f}, Kd={best_pids[i][2]:.3f}',
        fontsize=14,
        fontweight='bold'
    )
    ax.legend(fontsize=11, loc='best')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pv_vs_sp_agents.png', dpi=150, bbox_inches='tight')
print("📊 Gráfico guardado: pv_vs_sp_agents.png")
plt.show()

## 7. Visualización - Error por Agente

In [ ]:
# Crear gráfico de error para cada agente
fig, axes = plt.subplots(n_agents, 1, figsize=(12, 4*n_agents))

if n_agents == 1:
    axes = [axes]

for i in range(n_agents):
    ax = axes[i]
    traj = trajectories[i]
    dead_band = config['dead_band'][i]
    
    # Error
    ax.plot(traj['error'], 'r-', linewidth=2)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.5, linewidth=2)
    ax.axhline(y=dead_band, color='g', linestyle=':', alpha=0.5, label='Dead Band')
    ax.axhline(y=-dead_band, color='g', linestyle=':', alpha=0.5)
    
    # Configuración
    ax.set_xlabel('Step', fontsize=12)
    ax.set_ylabel('Error', fontsize=12)
    ax.set_title(f'Agente Controlador {i} - Error de Control', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('error_agents.png', dpi=150, bbox_inches='tight')
print("📊 Gráfico guardado: error_agents.png")
plt.show()

## 8. Visualización - Acciones de Control por Agente

In [ ]:
# Crear gráfico de acciones para cada agente
fig, axes = plt.subplots(n_agents, 1, figsize=(12, 4*n_agents))

if n_agents == 1:
    axes = [axes]

for i in range(n_agents):
    ax = axes[i]
    traj = trajectories[i]
    
    # Acción de control
    ax.plot(traj['actions'], 'purple', linewidth=2)
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.5)
    
    # Configuración
    ax.set_xlabel('Step', fontsize=12)
    ax.set_ylabel('Acción de Control', fontsize=12)
    ax.set_title(
        f'Agente Controlador {i} - Salida del PID',
        fontsize=14,
        fontweight='bold'
    )
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('actions_agents.png', dpi=150, bbox_inches='tight')
print("📊 Gráfico guardado: actions_agents.png")
plt.show()

## 9. Estadísticas Finales

In [ ]:
# Obtener estadísticas
stats = multi_env.get_statistics()

print("\n" + "="*80)
print("ESTADÍSTICAS FINALES")
print("="*80)

print("\nPIDTrainer:")
for key, value in stats['pid_trainer'].items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

print("\nStabilityCriteria:")
for key, value in stats['stability_criteria'].items():
    if isinstance(value, float):
        if 'rate' in key:
            print(f"  {key}: {value:.2%}")
        else:
            print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

print("\n" + "="*80)

## 10. Resumen de Performance por Agente

In [ ]:
print("\n" + "="*80)
print("RESUMEN DE PERFORMANCE POR AGENTE")
print("="*80)

for i in range(n_agents):
    traj = trajectories[i]
    
    # Calcular métricas
    final_error = abs(traj['error'][-1])
    max_error = max(abs(e) for e in traj['error'])
    avg_error = np.mean([abs(e) for e in traj['error']])
    
    # Tiempo de establecimiento (aprox)
    dead_band = config['dead_band'][i]
    settling_time = None
    for t, e in enumerate(traj['error']):
        if abs(e) <= dead_band:
            settling_time = t
            break
    
    print(f"\nAgente Controlador {i}:")
    print(f"  PID: Kp={best_pids[i][0]:.4f}, Ki={best_pids[i][1]:.4f}, Kd={best_pids[i][2]:.4f}")
    print(f"  Error final: {final_error:.4f}")
    print(f"  Error máximo: {max_error:.4f}")
    print(f"  Error promedio: {avg_error:.4f}")
    if settling_time is not None:
        print(f"  Tiempo de establecimiento: {settling_time} steps")
    else:
        print(f"  Tiempo de establecimiento: No alcanzado")
    
    # Éxito
    success = final_error <= dead_band
    print(f"  Estado: {'✅ ÉXITO' if success else '⚠️  FUERA DE BANDA'}")

print("\n" + "="*80)

## 11. (OPCIONAL) Modo Indirecto con Orquestador

Descomentar para probar modo indirecto:

In [ ]:
# # Configuración para MODO INDIRECTO
# config_indirect = {
#     'mode': 'indirect',
#     
#     # Variables manipulables
#     'n_manipulable_vars': 2,
#     'n_variables': 2,
#     
#     # Variables objetivo
#     'n_target_vars': 1,
#     'target_ranges': [(0.0, 10.0)],
#     'target_setpoints': [5.0],
#     
#     # Entrenamiento
#     'n_episodes': 30,
#     'j_max_retries': 3,
#     'r_orchestrator_iterations': 10,
#     
#     # Rangos operativos
#     'sp_ranges': [(40.0, 60.0), (35.0, 55.0)],
#     
#     # Parámetros ambiente
#     'upper_range': [100.0, 90.0],
#     'lower_range': [0.0, 0.0],
#     'setpoint': [50.0, 45.0],
#     'dead_band': [2.0, 2.0],
#     'dt': 1.0,
#     'max_episode_steps': 100,
#     
#     # Parámetros agentes
#     'agent_lr': 0.001,
#     'agent_gamma': 0.99,
#     'orch_lr_actor': 0.0001,
#     'orch_lr_critic': 0.001,
#     'orch_gamma': 0.99,
#     'device': 'cpu'
# }

# # Crear y entrenar
# multi_env_indirect = MultiAgentPIDEnv(config_indirect)
# best_pids_ind, best_sps = multi_env_indirect.train()

# print(f"\nMejores PIDs: {best_pids_ind}")
# print(f"Mejores Setpoints: {best_sps}")